In [2]:
import xlrd
import math as m


class nodo():
	def __init__(self,ide,nome,lat,lon,stato,identificativo,defi,pot,coll,adia): 
		self.ide=ide
		self.lat=lat
		self.lon=lon
		self.nome=nome
		self.stato=stato
		self.identificativo=identificativo
		self.defi=defi
		self.pot=pot
		self.coll=coll
		self.adia=adia


def dist2punti(latA,lonA,latB,lonB):
	Rt=6378.388
	lat_A = (m.pi*latA)/180.
	lat_B = (m.pi*latB)/180.
	lon_A = (m.pi*lonA)/180.
	lon_B = (m.pi*lonB)/180.
	dl=abs(lon_A-lon_B)

	p = m.acos((m.sin(lat_B)*m.sin(lat_A))+(m.cos(lat_B)*m.cos(lat_A)*m.cos(dl)))
	if(lon_A>0 and lon_B>0): 
		if(lon_B > lon_A): 
			p=p
		else:
			p=float('inf')
	if(lon_A<0 and lon_B<0): 
		if(lon_B > lon_A): 
			p=p
		else:
			p=float('inf')
	if(lon_A>0 and lon_B<0):
		if((lon_A+abs(lon_B))<m.pi): 
			p=float('inf')
		else:
			p=p
	if(lon_A<0 and lon_B>0):
		if((lon_B+abs(lon_A))<m.pi): 
			p=p
		else:
			p=float('inf')
	return(p*Rt) 


dati=xlrd.open_workbook('/Users/Tonino222power/Desktop/City2.xlsx')
foglio = dati.sheet_by_name('Foglio1')
nR=foglio.nrows
nC=foglio.ncols


citta_inizio="London"
inizio="1826645935"



nome=[]
lat=[]
log=[]
stato=[]
stato2=[]
pop=[]
iden=[]
pesi=[]
nodi=[]


for i in range(1,nR):	
	nome.append(foglio.cell(i,0).value)
	lat.append(foglio.cell(i,2).value)
	log.append(foglio.cell(i,3).value)
	stato.append(foglio.cell(i,6).value)
	stato2.append(foglio.cell(i,7).value)
	pop.append(foglio.cell(i,9).value)
	iden.append(str(int(foglio.cell(i,10).value)))

print("Lettura dati da file terminata")


for i in range(0,nR-1):
	if(pop[i]==''):
		pop[i]=0

print("Controllo popolazioni terminato")



elim=[]
elim1=[]

for i in range(0,nR-1):
	for j in range(0,nR-1):
		if(lat[j]==lat[i] and log[j]==log[i] and iden[j]!=iden[i]):
			print(j)
			elim.append(j)

for i in range(0,nR-1):
	for j in range(0,nR-1):
		if(iden[j]==iden[i] and lat[j]!=lat[i] and log[j]!=log[i]):
			print(k)
			elim1.appen(j)

for j in elim:
	print("rimosso per coordinate coincidenti: ",iden[j])
	nome.pop(j)
	lat.pop(j)
	log.pop(j)
	stato.pop(j)
	stato2.pop(j)
	pop.pop(j)
	iden.pop(j)

for j in elim1:
	print("rimosso per identificativo coincidente: ",iden[j])
	nome.pop(j)
	lat.pop(j)
	log.pop(j)
	stato.pop(j)
	stato2.pop(j)
	pop.pop(j)
	iden.pop(j)


print(len(nome))
nR=len(nome)

citta_fine=citta_inizio+"_fine"
fine="1111111111"

for i in range(0,nR):
	if(iden[i]==inizio):
		nome.append(citta_fine)
		lat.append(lat[i])
		log.append(log[i]-0.1)
		stato.append(stato[i])
		stato2.append(stato2[i])
		pop.append(pop[i])
		iden.append(fine)


print("Numerosità città")
print(len(nome))
nN=len(nome)


for i in range(0,nN):
	vicini=[]
	for j in range(0,nN):
		if(iden[i]==inizio and iden[j] == fine):
			continue
		if(iden[j]==inizio):
			continue
		if(iden[i]!=iden[j]):
			d=dist2punti(lat[i],log[i],lat[j],log[j])
			if(len(vicini)==0):
				vicini.append([iden[j],d,pop[j],stato[i]+"-"+stato2[i],stato[j]+"-"+stato2[j]])
			else:	
				for k in range(0,len(vicini)):
					if(d<vicini[k][1]):
						vicini.insert(k,[iden[j],d,pop[j],stato[i]+"-"+stato2[i],stato[j]+"-"+stato2[j]])
						if(len(vicini)>3):					
							vicini.pop(3)
						break
					if(k==2 and len(vicini)<3):
						vicini.append([iden[j],d,pop[j],stato[i]+"-"+stato2[i],stato[j]+"-"+stato2[j]])

	diz={}
	for l in range(0,len(vicini)):

		pes=2**(l+1)
		if(vicini[l][2]>200000):
			pes=pes+2

		if(vicini[l][3] != vicini[l][4]):
			pes=pes+2
		diz[vicini[l][0]]=pes 

	print("Ancora "+str(nN-i)+" citta' da controllare" )

	pesi.append(diz)

	nodi.append(nodo(i,nome[i],lat[i],log[i],stato[i]+"-"+stato2[i],iden[i],True,float('inf'),"",pesi[i])) 

print("Algoritmo di calcolo del grafo utilizzando l'algoritmo Dijkstra")



for x in nodi:
	if(x.identificativo == inizio):
		x.defi=False
		x.pot=0
		x.coll=inizio
		i=x.ide
k=0
while True:
	for x in nodi[i].adia.keys():
		for y in nodi:
			if (y.identificativo == x and y.defi ):
				somma=nodi[i].pot + nodi[i].adia[x]
				if(y.pot>somma):
					y.pot=somma
					y.coll=nodi[i].identificativo
	m=float('inf')
	for z in nodi:
		if(z.defi and z.pot<m):
			m=z.pot
			n=z.ide
	
	nodi[n].defi=False
	i=n
	print(nodi[i].nome, i)

	if(nodi[i].identificativo == fine ):
		lung=nodi[i].pot
		break
k=k+1

percorso=[]
percorso_nomi=[]


s=fine
while True:
	percorso.append(s)
	for x in nodi:
		if(x.identificativo == s):
			s=x.coll
	if(s==inizio):
		percorso.append(s)
		break


for x in percorso:
	for y in nodi:
		if(y.identificativo==x):
			percorso_nomi.append(y.nome+" ("+y.stato+")")

strada=""
for i in range(0,len(percorso_nomi)-1):
	strada=strada+percorso_nomi[len(percorso_nomi)-1-i]+" - "
strada=strada+" "+percorso_nomi[0]

print("-------------------Risultato----------------------------")
print("Percorso terminato in "+str(lung)+" ore o "+str(float(lung)/24.)+" giorni. (tempo minimo)" )
print("Tragitto: " + strada)
print("--------------------------------------------------------")

Lettura dati da file terminata
Controllo popolazioni terminato
3914
2943
3977
1155
183
1426
rimosso per coordinate coincidenti:  1484228590
rimosso per coordinate coincidenti:  1484980825
rimosso per coordinate coincidenti:  1418863956
rimosso per coordinate coincidenti:  1484504936
rimosso per coordinate coincidenti:  1484559591
rimosso per coordinate coincidenti:  1076190259
4994
Numerosità città
4995
Ancora 4995 citta' da controllare
Ancora 4994 citta' da controllare
Ancora 4993 citta' da controllare
Ancora 4992 citta' da controllare
Ancora 4991 citta' da controllare
Ancora 4990 citta' da controllare
Ancora 4989 citta' da controllare
Ancora 4988 citta' da controllare
Ancora 4987 citta' da controllare
Ancora 4986 citta' da controllare
Ancora 4985 citta' da controllare
Ancora 4984 citta' da controllare
Ancora 4983 citta' da controllare
Ancora 4982 citta' da controllare
Ancora 4981 citta' da controllare
Ancora 4980 citta' da controllare
Ancora 4979 citta' da controllare
Ancora 4978 cit

Ancora 4758 citta' da controllare
Ancora 4757 citta' da controllare
Ancora 4756 citta' da controllare
Ancora 4755 citta' da controllare
Ancora 4754 citta' da controllare
Ancora 4753 citta' da controllare
Ancora 4752 citta' da controllare
Ancora 4751 citta' da controllare
Ancora 4750 citta' da controllare
Ancora 4749 citta' da controllare
Ancora 4748 citta' da controllare
Ancora 4747 citta' da controllare
Ancora 4746 citta' da controllare
Ancora 4745 citta' da controllare
Ancora 4744 citta' da controllare
Ancora 4743 citta' da controllare
Ancora 4742 citta' da controllare
Ancora 4741 citta' da controllare
Ancora 4740 citta' da controllare
Ancora 4739 citta' da controllare
Ancora 4738 citta' da controllare
Ancora 4737 citta' da controllare
Ancora 4736 citta' da controllare
Ancora 4735 citta' da controllare
Ancora 4734 citta' da controllare
Ancora 4733 citta' da controllare
Ancora 4732 citta' da controllare
Ancora 4731 citta' da controllare
Ancora 4730 citta' da controllare
Ancora 4729 ci

Ancora 4512 citta' da controllare
Ancora 4511 citta' da controllare
Ancora 4510 citta' da controllare
Ancora 4509 citta' da controllare
Ancora 4508 citta' da controllare
Ancora 4507 citta' da controllare
Ancora 4506 citta' da controllare
Ancora 4505 citta' da controllare
Ancora 4504 citta' da controllare
Ancora 4503 citta' da controllare
Ancora 4502 citta' da controllare
Ancora 4501 citta' da controllare
Ancora 4500 citta' da controllare
Ancora 4499 citta' da controllare
Ancora 4498 citta' da controllare
Ancora 4497 citta' da controllare
Ancora 4496 citta' da controllare
Ancora 4495 citta' da controllare
Ancora 4494 citta' da controllare
Ancora 4493 citta' da controllare
Ancora 4492 citta' da controllare
Ancora 4491 citta' da controllare
Ancora 4490 citta' da controllare
Ancora 4489 citta' da controllare
Ancora 4488 citta' da controllare
Ancora 4487 citta' da controllare
Ancora 4486 citta' da controllare
Ancora 4485 citta' da controllare
Ancora 4484 citta' da controllare
Ancora 4483 ci

Ancora 4268 citta' da controllare
Ancora 4267 citta' da controllare
Ancora 4266 citta' da controllare
Ancora 4265 citta' da controllare
Ancora 4264 citta' da controllare
Ancora 4263 citta' da controllare
Ancora 4262 citta' da controllare
Ancora 4261 citta' da controllare
Ancora 4260 citta' da controllare
Ancora 4259 citta' da controllare
Ancora 4258 citta' da controllare
Ancora 4257 citta' da controllare
Ancora 4256 citta' da controllare
Ancora 4255 citta' da controllare
Ancora 4254 citta' da controllare
Ancora 4253 citta' da controllare
Ancora 4252 citta' da controllare
Ancora 4251 citta' da controllare
Ancora 4250 citta' da controllare
Ancora 4249 citta' da controllare
Ancora 4248 citta' da controllare
Ancora 4247 citta' da controllare
Ancora 4246 citta' da controllare
Ancora 4245 citta' da controllare
Ancora 4244 citta' da controllare
Ancora 4243 citta' da controllare
Ancora 4242 citta' da controllare
Ancora 4241 citta' da controllare
Ancora 4240 citta' da controllare
Ancora 4239 ci

Ancora 4021 citta' da controllare
Ancora 4020 citta' da controllare
Ancora 4019 citta' da controllare
Ancora 4018 citta' da controllare
Ancora 4017 citta' da controllare
Ancora 4016 citta' da controllare
Ancora 4015 citta' da controllare
Ancora 4014 citta' da controllare
Ancora 4013 citta' da controllare
Ancora 4012 citta' da controllare
Ancora 4011 citta' da controllare
Ancora 4010 citta' da controllare
Ancora 4009 citta' da controllare
Ancora 4008 citta' da controllare
Ancora 4007 citta' da controllare
Ancora 4006 citta' da controllare
Ancora 4005 citta' da controllare
Ancora 4004 citta' da controllare
Ancora 4003 citta' da controllare
Ancora 4002 citta' da controllare
Ancora 4001 citta' da controllare
Ancora 4000 citta' da controllare
Ancora 3999 citta' da controllare
Ancora 3998 citta' da controllare
Ancora 3997 citta' da controllare
Ancora 3996 citta' da controllare
Ancora 3995 citta' da controllare
Ancora 3994 citta' da controllare
Ancora 3993 citta' da controllare
Ancora 3992 ci

...

Mykolaiv 1363
Tambov 2042
Kamensk-Ural’skiy 3054
Kapaklı 4052
Nikopol’ 4148
Lysychansk 4435
Tekirdağ 693
Dnipro 835
Vladimir 1747
Çorlu 2203
Sievierodonetsk 3322
Neftekamsk 3815
Saransk 1932
Kherson 2047
Nizhnekamsk 2389
Pavlohrad 4342
Alchevsk 4355
Istanbul 20
Ufa 636
Tyumen 1006
Zaporizhzhia 1008
Penza 1312
Cheboksary 1389
Horlivka 2244
Kramatorsk 3221
Bandırma 3298
Miass 3354
Novocheboksarsk 3781
Khrustalnyi 3907
Yevpatoriya 4344
Chelyabinsk 590
Naberezhnyye Chelny 1297
Luhansk 1565
Kurgan 1901
Almetyevsk 3308
Oktyabr’skiy 4125
Kuznetsk 4762
Simferopol 1792
Zlatoust 3099
Melitopol’ 3288
Kopeysk 3423
Çayırova 3757
Bursa 270
Ulyanovsk 1158
Sterlitamak 2091
Yoshkar-Ola 2168
Qostanay 2557
Petropavl 2561
Syzran 3012
Körfez 3119
Kerch 3376
Yalova 3473
Berdyans’k 4113
Novoshakhtinsk 4314
Khanty-Mansiysk 4597
Kazan 572
Sevastopol 1541
Magnitogorsk 1567
Balakovo 2808
Gölcük 3160
Nefteyugansk 3856
Dimitrovgrad 4082
Donetsk 866
Tolyatti 1046
İnegöl 2172
Shakhty 2398
Novocherkassk 3071
Köksheta

Robāţ Karīm 4403
Shiyan 239
Kaiyuan 616
Majuro 808
Zhuozhou 1186
Shenzhou 1243
Shahe 1398
Kāshān 1536
Anguo 1703
Burnaby 2430
Karnāl 2487
Gurgaon 2695
Baoding 38
Handan 50
Hengshui 190
Xiaoxita 192
San Francisco 219
Indore 352
Zhangjiajie 493
New Delhi 784
Tarawa 806
Chengguan 1011
Shuangqiao 1019
Sahāranpur 1047
Mālegaon 1067
Yanshi Chengguanzhen 1100
Sanhe 1116
Dehra Dūn 1224
Bhīlwāra 1647
Pānīpat 2037
Richmond 2729
Shāhīn Shahr 3029
Yunxian Chengguanzhen 3628
Yāsūj 4327
Delta 4510
Alameda 4906
Langfang 178
Bhopāl 402
Farīdābād 522
Port Moresby 760
Honiara 794
Dengtalu 891
Bazhou 1163
Surrey 1326
Dhūlia 1421
Nangong 1431
Xiaoyi 1443
Huichang 1716
Ahmadnagar 1770
Najafābād 2407
Burhānpur 2739
Coquitlam 3592
Borāzjān 4266
Delhi 2
Changde 108
Nangandao 114
Jiaozuo 222
Eşfahān 427
Hebi 516
Kota 714
Funafuti 819
Renqiu 946
Hejian 958
Zijinglu 961
Zunhua 1015
Danjiangkou 1502
Oakland 1533
Muzaffarnagar 1630
Yidu 1663
Santa Rosa 1896
Khomeynī Shahr 2530
Vidisha 3272
Marvdasht 3411
Shahreẕā 

Negapatam 4460
Tejupilco 4875
Cheyenne 4876
Juventino Rosas 4910
Shenzhen 17
Huai’an 153
Meizhou 174
Wuhu 202
Weihai 304
Vishākhapatnam 381
Gaoyou 957
Guixi 1124
Xinxing 1154
Jurong 1169
Yong’an 1750
Irvine 2070
Ciudad Santa Catarina 2228
Birāṭnagar 2321
San Bernardino 2564
Ingrāj Bāzār 2642
Garden Grove 3049
Zinacantepec 3652
Chilapa de Álvarez 3818
Basīrhat 3833
Arvada 3968
Duluth 4002
Bānsbāria 4118
Highlands Ranch 4419
Kalyani 4451
San Luis de la Paz 4541
Tenancingo 4611
Westminster 4698
Temoaya 4869
Jieyang 112
Huizhou 199
Denver 278
Longyan 298
Dandong 332
Puning 378
Gongzhuling 649
Dongtai 832
San Luis Potosí 943
Wusong 1037
Mission Viejo 1207
Zhenzhou 1248
Liaoyuan 1324
San Nicolás de los Garza 1405
Rājshāhi 1481
Santa Ana 1846
Bāli 2035
Jessore 2329
Vizianagaram 2450
Moreno Valley 2591
Dam Dam 2962
Corona 3075
Bālurghāt 3292
Nawābganj 3505
Greeley 3618
Hālīsahar 3855
Darjeeling 3980
St. Cloud 4127
Centennial 4254
Lerma 4406
Jilotepec 4555
Jaffna 4678
Shangrao 87
Xuanzhou 307
S

Nakhon Sawan 4743
Kanasín 4751
Qujing 101
Rangoon 121
Yibin 171
Ulsan 611
Zhongba 989
Ch’ŏnan 1121
Wenlan 1574
Kaiyuan 1902
Asheville 1952
Gyeongsan 2112
Columbus 2222
Mawlamyine 2260
Hickory 2550
Gangneung 2563
Erie 2880
Spartanburg 2902
Fredericksburg 3086
Athens 3550
Yonashiro-teruma 3934
Milton 4279
State College 4735
Atlanta 120
Daegu 319
Shuangshui 584
Richmond 663
Vientiane 728
Taihe 871
Jiannan 1427
Tallahassee 2242
Quetzaltenango 2476
Oakville 2783
Gastonia 2905
Chetumal 3264
Warner Robins 3285
Mudon 3342
Chimaltenango 4779
Ban Talat Rangsit 4816
Centreville 4945
Xiping 1115
Brampton 1205
Kaihua 1417
Harrisburg 1506
Greensboro 1795
Kyŏngju 2198
Playa del Carmen 2237
Shinozaki 2243
Chishui 2394
Pak Kret 2828
Hagerstown 2877
Newport News 2951
Escuintla 3033
Alexandria 3223
Macon 3326
Rock Hill 4012
Cozumel 4787
Chongqing 58
Bijie 89
Charlotte 483
Kitaku 829
Mississauga 1034
Hamilton 1062
Kagoshima 1199
Greenville 1403
Xingren 1571
Concord 2216
Arlington 2438
Gainesville 2676
Bur

Altamira 4849
Poole 3317
Guasdualito 3807
Tinaquillo 4278
Tucuruí 4785
Belém 525
Shizuoka 1066
Sheffield 1073
Reading 1826
Luton 2554
Tocuyito 2732
Castanhal 3537
High Wycombe 3723
Hemel Hempstead 4519
Wakefield 4542
Marabá 2185
Fuji 2327
Bournemouth 2850
Saint Albans 3358
Ōmiyachō 3790
Basingstoke 4053
Barnsley 4644
Valencia 494
Slough 3138
Los Guayos 3732
Solihull 3853
Rotherham 4189
Mineshita 4287
Watford 4554
Bracknell 4726
São Luís 664
Southampton 917
Guacara 2775
Doncaster 4108
Edgware 4748
Uxbridge 4950
Leeds 396
Milton Keynes 2433
Imperatriz 2516
Kawara 2799
San Diego 2848
Harrow 3316
Hayes 4687
Codó 4730
Gosport 4969
Finchley 4985
Coventry 1721
Caxias 3596
El Limón 4335
Mansfield 4365
Nuneaton 4667
Bacabal 4900
Hiratsuka 2221
Enfield 3696
Wembley 4321
Isehara 4454
Southall 4932
Rugby 4943
Portsmouth 920
Maracay 1603
Timon 2595
Palo Negro 3307
Hounslow 4226
Harrogate 4885
London_fine 4994
-------------------Risultato----------------------------
Percorso terminato in 576 ore o 2